データサイエンス特別PG データサイエンス特論 第8回 課題

In [ ]:
# google colab で実行する場合は下記のコメントアウトを外して実行する
# ! wget https://github.com/KHiraGit/sudspg_ds/raw/main/ds07_temp_power_exercise.xlsx

In [1]:
import numpy as np
import pandas as pd
import openpyxl
import statsmodels.api as sm
import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [2]:
power_df = pd.read_excel('ds07_temp_power_exercise.xlsx', 
                         sheet_name='東京電力パワーグリッド エリア需給実績データ', skiprows=[0,1],
                         names=['DATE', 'TIME', 'power', '', '', '', '', '', '', '', '', '', '', '', ''])
power_df['DATETIME'] = pd.to_datetime(power_df['DATE']+' '+power_df['TIME'], format='%Y/%m/%d %H:%M')

In [3]:
weather_df = pd.read_excel('ds07_temp_power_exercise.xlsx', 
                           sheet_name='2020熊谷市気象データ_data', skiprows=[0,1,2],
                           names=['年月日', '平均気温', '', '', '最高気温', '', '', '', '', '最低気温', '', '', '', '', 
                           '降水量', '', '', '', '日照時間', '', '', '', '降雪量', '', '', '', '平均風速', '', '', 
                           '平均蒸気圧', '', '', '平均湿度', '', '', '平均現地気圧', '', '', '', '', '', '天気概況', '', ''])

In [4]:
year = 2020
month = 1
weather_data = ['平均気温', '最高気温', '最低気温', '降水量', '日照時間', '降雪量', 
                '平均風速', '平均蒸気圧', '平均湿度', '平均現地気圧'] 
print(f'## {year}年{month}月の気象データと電力需要の単回帰・多項式回帰分析')

# 対象期間・対象の気温の独立変数と応答変数を用意
x = weather_df[(datetime.datetime(year,month,1) <= weather_df['年月日']) 
        & (weather_df['年月日'] < datetime.datetime(year,month+1,1))][weather_data].values
y = power_df[(datetime.datetime(year,month,1) <= power_df['DATETIME'])
        & (power_df['DATETIME'] < datetime.datetime(year,month+1,1))
        & (power_df['TIME'] == '14:00')][['power']].values
x_added_constant = sm.add_constant(x)

# 重回帰分析を実施、独立変数間の分散拡大要因を確認後、分析結果を出力
model = sm.OLS(y, x_added_constant)
num_cols = model.exog.shape[1] # 説明変数の列数
vifs = [vif(model.exog, i) for i in range(0, num_cols)]
print(pd.DataFrame(vifs, index=model.exog_names, columns=['VIF']))
result = model.fit()
print(result.summary())

## 2020年1月の気象データと電力需要の単回帰・多項式回帰分析
                VIF
const  79967.299749
x1       104.321828
x2        22.566540
x3         5.061398
x4         5.260251
x5         8.659592
x6         5.396535
x7         4.936273
x8       170.514824
x9       178.980478
x10        3.504841
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     1.710
Date:                Thu, 09 Jun 2022   Prob (F-statistic):              0.147
Time:                        12:07:18   Log-Likelihood:                -230.71
No. Observations:                  31   AIC:                             483.4
Df Residuals:                      20   BIC:                             499.2
Df Model:                          10                                         
Covariance Type

In [5]:
# VIFが10以上の項目を削除して、独立変数間の分散拡大要因を再確認
year = 2020
month = 1
weather_data = ['最低気温', '降水量', '日照時間', '降雪量', '平均風速', '平均現地気圧'] 
print(f'## {year}年{month}月の気象データと電力需要の単回帰・多項式回帰分析')

# 対象期間・対象の気温の独立変数と応答変数を用意
x = weather_df[(datetime.datetime(year,month,1) <= weather_df['年月日']) 
        & (weather_df['年月日'] < datetime.datetime(year,month+1,1))][weather_data].values
x_added_constant = sm.add_constant(x)

# 重回帰分析を実施して結果を出力
model = sm.OLS(y, x_added_constant)

# 独立変数間の分散拡大要因を確認
num_cols = model.exog.shape[1] # 説明変数の列数
vifs = [vif(model.exog, i) for i in range(0, num_cols)]
print(pd.DataFrame(vifs, index=model.exog_names, columns=['VIF']))
result = model.fit()
print(result.summary())

## 2020年1月の気象データと電力需要の単回帰・多項式回帰分析
                VIF
const  31857.821876
x1         2.371454
x2         1.961314
x3         2.075784
x4         2.067238
x5         2.188736
x6         1.718988
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     2.551
Date:                Thu, 09 Jun 2022   Prob (F-statistic):             0.0470
Time:                        12:07:19   Log-Likelihood:                -232.64
No. Observations:                  31   AIC:                             479.3
Df Residuals:                      24   BIC:                             489.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                

In [11]:
import itertools
year = 2020
month = 1
weather_data = ['平均気温', '最高気温', '最低気温', '降水量', '日照時間', '降雪量', 
                '平均風速', '平均蒸気圧', '平均湿度', '平均現地気圧'] 

# 気象データの組み合わせを作成し、リストに格納
weather_data_combination = []
for n in range(1, len(weather_data)+1):
    for combination in itertools.combinations(weather_data, n):
        weather_data_combination.append(list(combination))  #タプルをリスト型に変換

# 気象データの組み合わせで重回帰分析を実施し、組み合わせとAICをdictに保存
combination_aic = {}
for combination in weather_data_combination:
    x = weather_df[(datetime.datetime(year,month,1) <= weather_df['年月日']) 
        & (weather_df['年月日'] < datetime.datetime(year,month+1,1))][combination].values
    x_added_constant = sm.add_constant(x)

    # 重回帰分析を実施し、
    model = sm.OLS(y, x_added_constant)
    result = model.fit()
    combination_aic['/'.join(combination)] = result.aic

# AICの小さい順から組み合わせを5パターン出力
combination_aic_sorted = sorted(combination_aic.items(), key=lambda x:x[1])
for i in range(10):
    print(i+1, combination_aic_sorted[i])

1 ('日照時間', 471.2777693352293)
2 ('日照時間/平均蒸気圧', 471.66398749995204)
3 ('日照時間/平均湿度', 471.84103767449153)
4 ('日照時間/降雪量', 472.0002044960938)
5 ('降水量/日照時間', 472.07707443740145)
6 ('日照時間/平均風速/平均湿度', 472.2249894135284)
7 ('最高気温/平均蒸気圧', 472.637196453738)
8 ('平均気温/日照時間', 472.81715772994494)
9 ('日照時間/降雪量/平均蒸気圧', 472.8734248440945)
10 ('最低気温/日照時間/平均風速/平均蒸気圧', 472.93939666472625)
